In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.join('..', 'src'))
import pandas as pd
from src.db_manager import SQLiteWrapper
from src.functions_notion import *

In [ ]:
sqlite_path = os.path.join("..", "data", "KoboReader.sqlite")

In [3]:
pd.options.display.max_columns = None

# Carga de BBDD

In [ ]:
db_path = os.path.join("..", "data", "KoboReader.sqlite")  # Ruta a tu archivo SQLite
db = SQLiteWrapper(db_path)
db.connect()  # Establecer conexión
anotaciones_df = db.get_annotations()
libros_ereader_df = db.get_books()
# db.close()

# Exploratorio tablas interesantes

In [ ]:
QUERY_ITEMS = """
    SELECT 
        b.VolumeID, 
        b.Text, 
        b.Annotation, 
        b.DateCreated, 
        b.DateModified,
        b.ChapterProgress, 
        b.Type as Type,
        c.BookTitle, 
        c.Title as Chapter, 
        c.Attribution as Author, 
        b.BookmarkID
    FROM Bookmark b INNER JOIN content c
    ON b.ContentID = c.ContentID 
    GROUP BY b.DateCreated 
    ORDER BY b.ChapterProgress ASC, b.DateCreated ASC;
"""

QUERY_ITEMS2 = """
    SELECT 
        b.VolumeID, 
        b.Text, 
        b.Annotation, 
        b.DateCreated, 
        b.DateModified,
        b.ChapterProgress, 
        b.Type as Type,
        c.BookTitle, 
        c.Title as Chapter, 
        c.Attribution as Author, 
        b.BookmarkID
    FROM Bookmark b INNER JOIN content c
    ON b.VolumeID = c.ContentID 
    GROUP BY b.DateCreated 
    ORDER BY b.ChapterProgress ASC, b.DateCreated ASC;
"""


QUERY_BOOKS = """
    SELECT DISTINCT
        b.VolumeID,
        c.BookTitle, 
        c.Title,
        c.Attribution as Author,
        (SELECT COUNT(*) FROM Bookmark b2 WHERE b2.VolumeID = b.VolumeID) AS Items
    FROM
        Bookmark b INNER JOIN content c ON b.VolumeID = c.ContentID
    ORDER BY
        c.Title;
"""

In [ ]:
content_df.astype(str).apply(lambda x: x.str.contains("arte de amar"))\
    .sum().loc[lambda x: x>0]

In [ ]:
content_df\
    .loc[lambda x: x.BookTitle.notna()]\
    .loc[lambda x: x.BookTitle.str.contains("arte de amar")]

In [ ]:
anotaciones_clean_df = anotaciones_df\
    .loc[lambda x: x.Type.isin(["highlight", "note"])]\
    [['BookTitle', 'VolumeID', 'Chapter', 'ChapterProgress', 'Text', 'Annotation', 'Type', 'DateCreated']]

In [ ]:
anotaciones_clean_df.head(1)

In [ ]:
libros_df.drop(["BookTitle", "Items"], axis = 1)

In [ ]:
content_df = db.get_query_df("""select * from content""")
nunique_cols_content = db.get_query_df("select * from content").nunique().loc[lambda x: x<= 1].index
content_df = content_df.drop(nunique_cols_content, axis = 1)
bookmark_df = db.get_query_df("""select * from Bookmark""")
nunique_cols_bookmark = db.get_query_df("select * from Bookmark").nunique().loc[lambda x: x<= 1].index
bookmark_df = bookmark_df.drop(nunique_cols_bookmark, axis = 1)

In [ ]:
anotaciones_df = db.get_query_df(QUERY_ITEMS)
anotaciones_df2 = db.get_query_df(QUERY_ITEMS2)

In [ ]:
anotaciones_df.isna().sum().loc[lambda x: x>0]

In [ ]:
anotaciones_df2.isna().sum().loc[lambda x: x>0]

In [ ]:
anotaciones_df2.shape

In [ ]:
anotaciones_df.head(1)

In [ ]:
anotaciones_df2.head(1)

In [ ]:
libros_df = db.get_query_df(QUERY_BOOKS)

In [ ]:
libros_df.head().VolumeID.values[0]

In [ ]:
QUERY_CHAPTERS = """
    SELECT DISTINCT
        b.VolumeID,
        c.BookTitle, 
        c.Title,
        c.Attribution as Author
    FROM
        Bookmark b INNER JOIN content c ON b.ContentID = c.ContentID
    ORDER BY
        c.Title;
"""
capitulos_df = db.get_query_df(QUERY_CHAPTERS)

In [ ]:
bookmark_df.head(1)

In [ ]:
content_df.loc[lambda x: x.BookTitle.notna()].loc[lambda x: x.BookTitle.str.contains("12")]\
    .nunique().loc
    # .iloc[0]

In [ ]:
db.get_query_df("""select * from Bookmark
                 where VolumeID like '%12_regl%' and Text is not Null
                  """)#[['VolumeID', 'ContentID']].VolumeID.unique()

In [ ]:
content: 
    ContentID: 'file:///mnt/onboard/.kobo/dropbox/12_reglas_para_vivir_Jordan_Peterson.epub#(0)OEBPS/Text/cubierta.xhtml'
    BookID: 'file:///mnt/onboard/.kobo/dropbox/12_reglas_para_vivir_Jordan_Peterson.epub'

Bookmark:
    VolumeID: 'file:///mnt/onboard/.kobo/dropbox/12_reglas_para_vivir_Jordan_Peterson.epub'
    ContentID: 'file:///mnt/onboard/.kobo/dropbox/12_reglas_para_vivir_Jordan_Peterson.epub#(7)OEBPS/Text/cap005.xhtml'


In [ ]:
capitulos_df

In [ ]:
anotaciones_df2.loc[lambda x: x.VolumeID.str.contains("12_reglas")].VolumeID.unique()

In [ ]:
anotaciones_df.loc[lambda x: x.VolumeID.str.contains("12_reglas")].VolumeID.unique()

In [ ]:
content_df.loc[lambda x: x.ContentID.str.contains("12_reglas")]

In [ ]:
bookmark_df

## Exploratorio de Todas las Tablas

In [ ]:
db_path = "KoboReader.sqlite"  # Ruta a tu archivo SQLite
db = SQLiteWrapper(db_path)
db.connect()  # Establecer conexión

In [ ]:
tables_info_df = db.get_tables_info() 

Tablas interesantes:
* content: contenido interesante
* Event
* Bookmark

Otras tablas:
* Shelf: Colecciones de libros. Por ejemplo, Histórica, Thriller, Filosofía...
* ShelfContent: Contenidos de las colecciones de Shelf
* WordList: Lista de palabras guardadas
* DropboxItem: Contenidos de dropbox
* user: Información del usuario. Revisar
* content_settings: configuración de lectura de varios libros
* Achievement: Premios de lectura

In [ ]:
tables_info_df.sort_values("num_filas")

In [ ]:
db.get_query_df("select * from content limit 10")

In [ ]:
db.get_query_df("select * from DbVersion")

In [ ]:
db.get_query_df("select * from volume_shortcovers limit 10 ")

In [ ]:
db.get_query_df("select * from user")

In [ ]:
db.get_query_df("select * from Tab")

In [ ]:
db.get_query_df("select * from KoboPlusAssetGroup")

In [ ]:
db.get_query_df("select * from GDriveItem")

In [ ]:
db.get_query_df("select * from Shelf")

In [ ]:
db.get_query_df("select * from AnalyticsEvents")

In [ ]:
db.get_query_df("select * from SubscriptionProducts")

In [ ]:
db.get_query_df("select * from content_settings limit 5")

In [ ]:
db.get_query_df("select * from Achievement")\
        .loc[lambda x: x.PercentComplete= 100].CompleteDescription.values

In [ ]:
db.get_query_df("select * from Rules limit 5")

In [ ]:
db.get_query_df("select * from ShelfContent limit 5")

In [ ]:
db.get_query_df("select * from WordList ")

In [ ]:
db.get_query_df("select * from Activity ")

In [ ]:
db.get_query_df("select * from DropboxItem").Json[0]

In [ ]:
db.get_query_df("select * from KoboPlusAssets where Language = 'es' ").Value.values

In [ ]:
db.get_query_df("select * from Event limit 10 ")

In [ ]:
db.get_query_df("select * from Bookmark limit 10 ")

In [ ]:
db.get_query_df("select * from volume_shortcovers limit 10 ")

In [ ]:
db.get_query_df("""select *
                from content limit 10
                """)

In [ ]:
db.get_query_df("""select BookID, BookTitle, count(*)
                from content 
                group by 1,2
                order by 3 desc""")

In [ ]:
tables_info_df.sort_values("num_filas")